In [ ]:
import numpy as np 
import pandas as pd 

### Append exp1, exp2 & exp3 

In [ ]:
exp1 = pd.read_csv('experiment_vol1.csv')
exp2 = pd.read_csv('experiment_vol2.csv')
exp3 = pd.read_csv('experiment_vol3.csv')           

In [ ]:
print(exp1.shape)
print(exp2.shape)
print(exp3.shape)

In [ ]:
exp_data = pd.concat([exp1,exp2,exp3])
exp_data.head()

In [ ]:
exp_data.isna().sum()

In [ ]:
exp_data.shape

### Append geom1, geom2 & geom3

In [ ]:
geom1 = pd.read_csv('geom_vol1.csv')
geom2 = pd.read_csv('geom_vol2.csv')
geom3 = pd.read_csv('geom_vol3.csv')

In [ ]:
geom_data = pd.concat([geom1,geom2,geom3],axis=0)
geom_data.head()

In [ ]:
geom_data.shape

In [ ]:
geom_data.isna().sum()

#### Changing exp data columns name

In [ ]:
exp_data.columns

In [ ]:
exp_data.columns = exp_data.columns.str.replace(' ','_')
exp_data.columns = exp_data.columns.str.replace("'",'')

In [ ]:
exp_data.columns

In [ ]:
exp_data.head()

### Changing geom data columns name

In [ ]:
geom_data.columns

In [ ]:
geom_data.columns = geom_data.columns.str.replace(' ','_')
geom_data.columns = geom_data.columns.str.replace("'",'')

In [ ]:
geom_data.columns

In [ ]:
geom_data.head()

### To Find Chord Distribution & Radius Distribution 

In [ ]:
geom_data['Chord_distribution'] = (geom_data['Propellers_Diameter'] / 2) * geom_data['Adimensional_Chord_-_c/R']
geom_data['Radius_distribution'] = (geom_data['Propellers_Diameter'] / 2) * geom_data['Adimensional_Radius_-_r/R']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.histplot(geom_data['Radius_distribution'],kde = True)
plt.show()

In [ ]:
sns.histplot(geom_data['Chord_distribution'],kde = True)
plt.show()

### Calculate Blade Area & Total Blade Area

In [ ]:
grouped_data = geom_data.groupby('Blades_Name')
results = []
for name, group in grouped_data:
    group = group.sort_values('Radius_distribution')
    blade_area = np.trapezoid(group['Chord_distribution'],group['Radius_distribution'])
    results.append({
        'Blades_Name':name,
        'Blade_Area':blade_area,
        'Total_Blade_Area': blade_area})
area_df = pd.DataFrame(results)    
geom_data = pd.merge(geom_data,area_df)
geom_data.head()

### Calculate Disc Area

In [ ]:
geom_data['Disc_Area'] = np.pi * (geom_data['Propellers_Diameter'] / 2) ** 2
geom_data.head()

### Calculate Solidity Value

In [ ]:
geom_data['Solidity'] = geom_data['Blade_Area'] / geom_data['Disc_Area']
geom_data.head()

In [ ]:
geom_data[['Blades_Name','Solidity']]

### Calculate for Experiment Data (Solidity,Disc Area, Blade Area, Total Blade Area)

In [ ]:
geom_group = geom_data.groupby('Blades_Name') 
for name, group in geom_group:
    group = group.sort_values('Radius_distribution')
    blade_area = np.trapezoid(group['Chord_distribution'],group['Radius_distribution'])
    results.append({
        'Blades_Name':name,
        'Blade_Area':blade_area,
        'Total_Blade_Area':blade_area})
area_df = pd.DataFrame(results)
exp = pd.merge(exp_data,area_df)
exp['Total_Blade_Area'] = exp['Blade_Area'] * exp['Number_of_Blades']
exp['Disc_Area'] = np.pi * (geom_data['Propellers_Diameter'] / 2) ** 2
exp['Solidity'] = exp['Blade_Area'] / exp['Disc_Area']
exp.head()

### Distribution of Disc Area, Solidity, Blade Area

In [ ]:
sns.histplot(exp['Disc_Area'])
plt.show()

In [ ]:
sns.histplot(exp['Solidity'])
plt.show()

In [ ]:
sns.histplot(exp['Total_Blade_Area'])
plt.show()

### Correlation Between Power Coefficient Output & Thrust Coefficient Output 

In [ ]:
sns.scatterplot(data = exp,x = 'Power_Coefficient_Output',y = 'Thrust_Coefficient_Output')
plt.show()

### Correlation Between Propellers Diameter & Pitch

In [ ]:
sns.scatterplot(data = exp,x = 'Propellers_Diameter',y = 'Propellers_Pitch')
plt.show()

### Count of propellers with no. of Blades

In [ ]:
num_blade = exp['Number_of_Blades'].value_counts()
sns.barplot(num_blade)
for i,val in enumerate(num_blade.index):
    plt.annotate(f"{num_blade[val]}".center(7), xy = (i - 0.15,num_blade[val]),size = 15,color = 'green')
plt.show()

### Propellers Brand with number of blades

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(data = exp,x = 'Propellers_Brand',y = 'Number_of_Blades')
plt.show()

### Heatmaps/ Correlation between variables

In [ ]:
corr = exp.corr(numeric_only=True)

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(corr,annot=True,fmt = '.2f')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
exp_corr = exp_data.corr(numeric_only=True).round(2)
exp_corr

In [ ]:
m = np.ones_like(exp_corr)
lower_tril = np.tril_indices_from(exp_corr)
m[lower_tril] = 0
m

In [ ]:
plt.figure(figsize = (8,4))
sns.heatmap(exp_corr,annot=True,fmt='.2f',mask = m,cbar = False)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
geom_corr = geom_data.corr(numeric_only=True)
geom_corr

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(geom_corr,annot=True,fmt = '.2f')
plt.xticks(rotation = 45)
plt.show()